In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [15]:
from sklearn.model_selection import train_test_split

In [4]:
sat = pd.read_csv("/Users/darshmac/Documents/cdac/For DBDA/PML/Cases/Satellite Imaging/Satellite.csv",delimiter=';')

In [5]:
sat

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,classes
0,92,115,120,94,84,102,106,79,84,102,...,104,88,121,128,100,84,107,113,87,grey soil
1,84,102,106,79,84,102,102,83,80,102,...,100,84,107,113,87,84,99,104,79,grey soil
2,84,102,102,83,80,102,102,79,84,94,...,87,84,99,104,79,84,99,104,79,grey soil
3,80,102,102,79,84,94,102,79,80,94,...,79,84,99,104,79,84,103,104,79,grey soil
4,84,94,102,79,80,94,98,76,80,102,...,79,84,103,104,79,79,107,109,87,grey soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,60,83,96,85,64,87,100,88,64,83,...,92,66,87,108,89,63,83,104,85,red soil
6431,64,79,100,85,56,71,96,85,56,68,...,85,66,83,100,85,63,83,100,81,red soil
6432,56,68,91,81,56,64,91,81,53,64,...,81,59,87,96,81,63,83,92,74,vegetation stubble
6433,56,68,87,74,60,71,91,81,60,64,...,74,59,83,92,74,59,83,92,70,vegetation stubble


In [7]:
X = sat.drop('classes',axis=1)

In [8]:
y = sat['classes']

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_y = le.fit_transform(y)
print(le.classes_)

['cotton crop' 'damp grey soil' 'grey soil' 'red soil'
 'vegetation stubble' 'very damp grey soil']


In [13]:
#Print the mapping of labels to categories
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

Label Mapping: {'cotton crop': 0, 'damp grey soil': 1, 'grey soil': 2, 'red soil': 3, 'vegetation stubble': 4, 'very damp grey soil': 5}


In [33]:
X_train,X_test,y_train,y_test=train_test_split(X,le_y,test_size=0.3,stratify=le_y,random_state=23)

In [75]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=23)
dtc.fit(X_train,y_train)

DecisionTreeClassifier(random_state=23)

In [76]:
y_pred = dtc.predict(X_test)

In [77]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [78]:
from sklearn.metrics import accuracy_score,log_loss

In [79]:
params = {'min_samples_split':[2,3,4,5,6,100],
         'max_depth':[None, 2,3,4,5,6,7],
         'min_samples_leaf':[1,2,3,4,5,6,10]}

In [80]:
gcv = GridSearchCV(dtc,param_grid=params,cv=kfold,scoring='neg_log_loss')

In [81]:
gcv.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=23),
             param_grid={'max_depth': [None, 2, 3, 4, 5, 6, 7],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 10],
                         'min_samples_split': [2, 3, 4, 5, 6, 100]},
             scoring='neg_log_loss')

In [82]:
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 100}
-0.6286473855672835


In [83]:
bst_model = gcv.best_estimator_

In [84]:
bst_model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=10, min_samples_split=100,
                       random_state=23)

In [85]:
y_pred = bst_model.predict_proba(X_test)

In [86]:
log_loss(y_test,y_pred)

0.6196914674978722

In [87]:
y_test.sum()

5495

In [88]:
y_pred.sum()

1931.0

In [89]:
y_test.shape

(1931,)

In [90]:
y_pred.shape

(1931, 6)

In [91]:
bst_model.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 10,
 'min_samples_split': 100,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 23,
 'splitter': 'best'}